# YOLO で転移学習

## インポート

In [ ]:
from pathlib import Path

import certifi
import numpy as np
from PIL import Image
from yolov4.tf import YOLOv4, SaveWeightsCallback
import tensorflow as tf

## 転移学習で使用する学習済みモデルのダウンロード

In [ ]:
# ダウンロードに失敗しないようにするためのおまじない
os.environ["SSL_CERT_FILE"] = certifi.where()

yolov4_tiny_weights_path = tf.keras.utils.get_file(
    fname="yolov4-tiny.conv.29",
    origin="https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29",
    cache_subdir="models/yolov4",
)
yolov4_tiny_weights_path

## 学習データを作成

labelImg でアノテーションを行い、下記のような構成で保存しておきます。
アノテーションの保存書式は YOLO を使用します。

```
dataset
├── classes.txt
├── test
│  ├── 0001.jpg
│  ├── 0001.txt
│  ├── 0002.jpg
│  ├── 0002.txt
│  ├── 0003.jpg
│  ├── 0003.txt
│  ├── 0004.jpg
│  ├── 0004.txt
│  ├── 0005.jpg
│  └── image_path.txt
└── train
   ├── 0001.jpg
   ├── 0001.txt
   ├── 0002.jpg
   ├── 0002.txt
   ├── 0003.jpg
   ├── 0003.txt
   ├── 0004.jpg
   ├── 0004.txt
   ├── 0005.jpg
   └── image_path.txt
```

**classes.txt**

アノテーションに対するラベル名を行ごとに定義したファイルです。

**サンプル:**

```
person
bicycle
car
motorbike
aeroplane
...
```

**0001.txt, 0002.txt,...**

YOLO 形式で保存されたバウンディングボックスの座標です。
labelImg でアノテーションを行い、YOLO 形式で保存すれば作成できます。
.jpg のファイル名に対応したアノテーションは同名の .txt に保存します。

各列の意味は `<label> <x_center> <y_center> <width> <height>` です。

- `<label>`: `classes.txt` の何行目のラベル名に対応するかを 0 始まりで指定
- `<x_center>`: バウンディングボックスの中心 x 座標を画像の幅で割った値
- `<y_center>`: バウンディングボックスの中心 y 座標を画像の高さで割った値
- `<width>`: バウンディングボックスの幅を画像の幅で割った値
- `<height>`: バウンディングボックスの高さを画像の高さで割った値

**サンプル:**
```
0 0.651231 0.532031 0.132474 0.201563
```

**image_path.txt**

画像データのパスを記述したファイルです。

**サンプル:**

```
0001.jpg
0002.jpg
0003.jpg
0004.jpg
0005.jpg
```

## 学習済みモデルをロード

In [ ]:
yolo = YOLOv4(tiny=True)
yolo.classes = "dataset/classes.txt"
yolo.input_size = 608
yolo.batch_size = 5

yolo.make_model()
yolo.load_weights(yolov4_tiny_weights_path, weights_type="yolo")

## 学習データと検証データをロード

`image_path.txt` 内の画像データのパスは `image_path_prefix` からの相対パスとみなされます。

In [ ]:
train_data = yolo.load_dataset(
    "dataset/train/image_path.txt",
    dataset_type="yolo",
    image_path_prefix="dataset/train",
    label_smoothing=0.05,
    training=True,
)
validation_data = yolo.load_dataset(
    "dataset/test/image_path.txt",
    dataset_type="yolo",
    image_path_prefix="dataset/test",
    training=False,
)

## ハイパーパラメータを定義

In [ ]:
epochs = 100
learning_rate = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

yolo.compile(optimizer=optimizer, loss_iou_type="ciou")

In [ ]:
def create_learning_rate_scheduler(epochs):
    def scheduler(epoch, lr):
        if epoch < int(epochs * 0.5):
            return lr
        elif epoch < int(epochs * 0.8):
            return lr * 0.5
        elif epoch < int(epochs * 0.9):
            return lr * 0.1
        else:
            return lr * 0.01

    return scheduler

## 学習を開始

In [ ]:
yolo.fit(
    train_data,
    epochs=epochs,
    validation_data=validation_data,
    validation_steps=5,
    validation_freq=10,
    steps_per_epoch=5,
    callbacks=[
        tf.keras.callbacks.LearningRateScheduler(create_learning_rate_scheduler(epochs)),
        tf.keras.callbacks.TerminateOnNaN(),
        tf.keras.callbacks.TensorBoard(log_dir="logs"),
        SaveWeightsCallback(
            yolo=yolo,
            dir_path="weights",
            weights_type="yolo",
            epoch_per_save=10,
        ),
    ],
)

## 学習したモデルを使用

In [ ]:
yolo.load_weights("weights/yolov4-tiny-final.weights", weights_type="yolo")

with open("dataset/test/image_path.txt") as f:
    output_images = []

    for image_path in map(lambda x: f"dataset/test/{x.strip()}", f):
        print(image_path)

        image = Image.open(image_path)
        image = np.array(image)

        boxes = yolo.predict(image)
        output_image = yolo.draw_bboxes(image, boxes)
        output_image = Image.fromarray(output_image)

        output_images.append(output_image)

    output_images[0].save("out.gif", save_all=True, append_images=output_images[1:], loop=0)